# Notebook zum Erzeugen eines IP-Blocks aus einem quantisierten neuronalen Netz
Dieses Jupyter-Notebook erzeugt aus einem quantisierten neuronalen Netz im ONNX-Format einen IP-Block, welcher in die Vivado Design Suite eingebunden werden kann.
Da dieses Notebook das FINN-Framework verwendet, muss es in einem Docker-Container ausgeführt werden, welcher die Abhängigkeiten zu FINN enthält.

Dieses Notebook stellt ein Proof of Concept zum Erzeugen eines IP Blocks dar, da das endgültige neuronale Netz des Projektes zum Zeitpunkt des Schreibens dieser Dokumentation noch nicht vorhanden war.

## Haupt-Dependencies importieren

In [1]:
import onnx
import torch

## Laden des neuronalen Netzes aus einer ONNX-Datei
Das neuronale Netz wird mithilfe der Klasse `ModelWrapper` in FINN als Modell geladen.

In [2]:
import os
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

# Das Verzeichnis der ONNX-Datei
model_dir = "."
# Das relative Verzeichnis für die Ausgabegabedatein des Notebooks
builder_dir = model_dir + "/builder_output"
# Der Dateiname der ONNX-Datei
ready_model_filename = model_dir + "/DefaultModel.onnx"

In [4]:
from finn.util.visualization import showInNetron, showSrc

# ModelWrapper wird zum Laden der ONNX-Datei in FINN benoetigt
# Zudem werden hierdurch verschiedene Hilsfunktionen fuer das Modell zur Verfuegung gestellt
model = ModelWrapper(ready_model_filename)

# Visualisierung des neuronalen Netzes
showInNetron(ready_model_filename)

Stopping http://0.0.0.0:8081
Serving './DefaultModel.onnx' at http://0.0.0.0:8081


## Notwendige Transformationen auf das Modell anwenden
Für die weiteren Verarbeitungsschritte müssen einige Transformationen auf das importierte Modell angewandt werden.

In [5]:
from qonnx.transformation.general import GiveUniqueNodeNames, GiveReadableTensorNames, RemoveStaticGraphInputs
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants

model = model.transform(InferShapes()) # Ableiten der Tensoren-Formen aus den Modell-Eigenschaften
model = model.transform(FoldConstants()) # Faltung von konstanten Netzwerkteilen => Vereinfachung des Netzwerks
model = model.transform(GiveUniqueNodeNames()) # Vergabe von eindeutigen Bezeichnern fuer die Netzwerkt-Nodes
model = model.transform(GiveReadableTensorNames()) # Vergabe von fuer Menschen lesbare Namen fuer Tensoren
model = model.transform(InferDataTypes()) # Ableiten der Tensoren-Datentypen aus den Modell-Eigenschaften
model = model.transform(RemoveStaticGraphInputs()) # Entfernen von Netzwerk-Eingaengen, welche schon ONNX-Initialisierer verwenden

# Speichern des transformierten Modells als Zwischenausgabe
model.save(builder_dir + "/model_tidy.onnx")

In [6]:
# Visualisierung des transfomierten Modells
model = ModelWrapper(builder_dir + "/model_tidy.onnx")
showInNetron(builder_dir + "/model_tidy.onnx")

Stopping http://0.0.0.0:8081
Serving './builder_output/model_tidy.onnx' at http://0.0.0.0:8081


## Umwandlung der Netzwerk-Layer in Hardware-Layer

In [7]:
from finn.transformation.fpgadataflow.create_dataflow_partition import CreateDataflowPartition

model = ModelWrapper(builder_dir + "/model_tidy.onnx")
parent_model = model.transform(CreateDataflowPartition())
parent_model.save(builder_dir + "/parent.onnx")

In [8]:
# Visualisierung des Modells mit Hardware-Layern
model = ModelWrapper(builder_dir + "/parent.onnx")
showInNetron(builder_dir + "/parent.onnx")

Stopping http://0.0.0.0:8081
Serving './builder_output/parent.onnx' at http://0.0.0.0:8081


## Finale Vorbereitung des Modells für die IP-Block-Erstellung

In [9]:
# Auflistung der verfuegbaren FPGA-Boards
from finn.util.basic import pynq_part_map
print(pynq_part_map.keys())

dict_keys(['Ultra96', 'Ultra96-V2', 'Pynq-Z1', 'Pynq-Z2', 'ZCU102', 'ZCU104', 'ZCU111', 'RFSoC2x2', 'RFSoC4x2', 'KV260_SOM'])


In [10]:
# FPGA-Board festlegen
pynq_board = "Pynq-Z2"
# FPGA-Board-Informationen auswaehlen
fpga_part = pynq_part_map[pynq_board]
# Takt-Frequenz festlegen
target_clk_ns = 10

In [11]:
from finn.transformation.fpgadataflow.make_zynq_proj import ZynqBuild
model = ModelWrapper(builder_dir + "/parent.onnx")

# Ausgabe der einzelnen Netzwerk-Nodes
for node in model.graph.node:
    print(node)

input: "global_in"
input: "MatMul_0_param0"
output: "MatMul_0_out0"
name: "MatMul_0"
op_type: "MatMul"
domain: ""

input: "MatMul_0_out0"
input: "Mul_0_param0"
output: "Mul_0_out0"
name: "Mul_0"
op_type: "Mul"

input: "Mul_0_out0"
input: "MultiThreshold_0_param0"
output: "MultiThreshold_0_out0"
name: "MultiThreshold_0"
op_type: "MultiThreshold"
attribute {
  name: "out_dtype"
  s: "UINT4"
  type: STRING
}
domain: "qonnx.custom_op.general"

input: "MultiThreshold_0_out0"
input: "Mul_1_param0"
output: "Mul_1_out0"
name: "Mul_1"
op_type: "Mul"

input: "Mul_1_out0"
input: "MultiThreshold_1_param0"
output: "MultiThreshold_1_out0"
name: "MultiThreshold_1"
op_type: "MultiThreshold"
attribute {
  name: "out_dtype"
  s: "UINT4"
  type: STRING
}
domain: "qonnx.custom_op.general"

input: "MultiThreshold_1_out0"
input: "Mul_2_param0"
output: "Mul_2_out0"
name: "Mul_2"
op_type: "Mul"

input: "Mul_2_out0"
input: "MatMul_1_param0"
output: "MatMul_1_out0"
name: "MatMul_1"
op_type: "MatMul"
domain: ""


In [12]:
from finn.transformation.fpgadataflow.prepare_ip import PrepareIP

# Vorbereitung des Modells zur Implementierung in einem IP-Block
model = ModelWrapper(builder_dir + "/parent.onnx")
model = model.transform(PrepareIP(fpga_part, target_clk_ns))
model.save(builder_dir + "/model_prepared.onnx")

In [13]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

# Umwandlung des Modells in ein fuer FINN und die weiteren Schritte notwendiges Format
model_file = builder_dir + "/model_prepared.onnx"
model = ModelWrapper(model_file)
model = model.transform(ConvertQONNXtoFINN())
model.save(builder_dir + "/model_file_ready.onnx")

## Erstellung des IP-Blocks

In [14]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

# Laden des fuer die IP-Block umgewandelten Modells
model_file = builder_dir + "/model_file_ready.onnx"
model = ModelWrapper(model_file)
rtl_output = model_dir + "/rtl_output"

# Loeschen von vorherigen Ausgaben des Build-Prozesses
if os.path.exists(rtl_output):
    shutil.rmtree(rtl_output)
    print("Previous run results deleted!")

# Erstellen der Konfiguration zur Erzeugung des IP-Blocks
cfg_stitched_ip = build.DataflowBuildConfig(
    output_dir          = rtl_output,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = target_clk_ns,
    fpga_part           = fpga_part,
    generate_outputs=[
        build_cfg.DataflowOutputType.STITCHED_IP,
    ],
    auto_fifo_depths = False,
    stitched_ip_gen_dcp = True
)

Previous run results deleted!


In [15]:
%%time
# Starten des Build-Prozesses fuer den IP-Block
build.build_dataflow_cfg(
    model_file, 
    cfg_stitched_ip,
)

Building dataflow accelerator from ./builder_output/model_file_ready.onnx
Intermediate outputs will be generated in /tmp/finn_dev_root
Final outputs will be generated in ./rtl_output
Build log is at ./rtl_output/build_dataflow.log
Running step: step_qonnx_to_finn [1/19]
Running step: step_tidy_up [2/19]
Running step: step_streamline [3/19]
Running step: step_convert_to_hw [4/19]
Running step: step_create_dataflow_partition [5/19]
Running step: step_specialize_layers [6/19]
Running step: step_target_fps_parallelization [7/19]
Running step: step_apply_folding_config [8/19]
Running step: step_minimize_bit_width [9/19]
Running step: step_generate_estimate_reports [10/19]
Running step: step_hw_codegen [11/19]
Running step: step_hw_ipgen [12/19]
Running step: step_set_fifo_depths [13/19]
Running step: step_create_stitched_ip [14/19]
Running step: step_measure_rtlsim_performance [15/19]
Running step: step_out_of_context_synthesis [16/19]
Running step: step_synthesize_bitfile [17/19]
Running s

0